In [6]:
import pandas as pd

def load_data():

    csv_path = "adult.csv"
    return pd.read_csv(csv_path)

data = load_data()


In [7]:
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [8]:
import numpy as np

data = data.replace('?',np.nan)

male_set = data[(data['gender'] == 'Male')].copy()
female_set = data[(data['gender'] == 'Female')].copy()

In [9]:
print(f"male {len(data[(data['gender'] == 'Male') & (data['income'] == '>50K')]) / len(data[(data['gender'] == 'Male')])}")
print(f"female {len(data[(data['gender'] == 'Female') & (data['income'] == '>50K')]) / len(data[(data['gender'] == 'Female')])}")

male 0.3037672281776417
female 0.10925148221343874


Cleaning Data

In [10]:
def clean_data(data):

  data = data.replace('?',np.nan)

  data = data.join(pd.get_dummies(data['workclass'], prefix='workclass'))
  data = data.drop(['workclass'], axis=1)

  data = data.drop(['education'], axis=1) #data already encoded by educational_num

  data['marital-status'] = data['marital-status'].replace(dict.fromkeys(['Never-married'], 'unmarried'))
  data['marital-status'] = data['marital-status'].replace(dict.fromkeys(['Married-civ-spouse', 'Married-AF-spouse'], 'married'))
  data['marital-status'] = data['marital-status'].replace(dict.fromkeys(['Separated', 'Married-spouse-absent'], 'separated'))
  data['marital-status'] = data['marital-status'].replace(dict.fromkeys(['Divorced'], 'divorced'))
  data['marital-status'] = data['marital-status'].replace(dict.fromkeys(['Widowed'], 'widowed'))
  data = data.join(pd.get_dummies(data['marital-status'], prefix='marital_status'))
  data = data.drop(['marital-status'], axis=1)

  data = data.join(pd.get_dummies(data['occupation'], prefix='occupation'))
  data = data.drop(['occupation'], axis=1)

  data = data.join(pd.get_dummies(data['relationship'], prefix='relationship'))
  data = data.drop(['relationship'], axis=1)

  data['race'] = data['race'].replace(dict.fromkeys(['Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other'], 0))
  data['race'] = data['race'].replace(['White'], [1])

  data['gender'] = data['gender'].replace(['Female','Male'], [0,1])

  data['native-country'].loc[(data['native-country'] != 'United-States')]='ExPat'
  data['native-country'] = data['native-country'].replace(['United-States', 'ExPat'], [0,1])

  data['income'] = data['income'].replace(['<=50K', '>50K'], [0,1])

  return data

data = clean_data(data.copy())

data.head()

C:\Users\bahab\AppData\Local\Temp\ipykernel_15608\1184947390.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['race'] = data['race'].replace(['White'], [1])
C:\Users\bahab\AppData\Local\Temp\ipykernel_15608\1184947390.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['gender'] = data['gender'].replace(['Female','Male'], [0,1])
C:\Users\bahab\AppData\Local\Temp\ipykernel_15608\1184947390.py:29: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignm

,age,fnlwgt,educational-num,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,...,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife
0,25,226802,7,0,1,0,0,40,0,0,...,False,False,False,False,False,False,False,True,False,False
1,38,89814,9,1,1,0,0,50,0,0,...,False,False,False,False,True,False,False,False,False,False
2,28,336951,12,1,1,0,0,40,0,1,...,True,False,False,False,True,False,False,False,False,False
3,44,160323,10,0,1,7688,0,40,0,1,...,False,False,False,False,True,False,False,False,False,False
4,18,103497,10,1,0,0,0,30,0,0,...,False,False,False,False,False,False,False,True,False,False


In [11]:
data.corrwith(data['income'])

age                             0.230369
fnlwgt                         -0.006339
educational-num                 0.332613
race                            0.083710
gender                          0.214628
capital-gain                    0.223013
capital-loss                    0.147554
hours-per-week                  0.227687
native-country                 -0.032551
income                          1.000000
workclass_Federal-gov           0.062112
workclass_Local-gov             0.034576
workclass_Never-worked         -0.008026
workclass_Private              -0.075625
workclass_Self-emp-inc          0.139596
workclass_Self-emp-not-inc      0.027190
workclass_State-gov             0.013619
workclass_Without-pay          -0.007002
marital_status_divorced        -0.128335
marital_status_married          0.446292
marital_status_separated       -0.083920
marital_status_unmarried       -0.318782
marital_status_widowed         -0.065050
occupation_Adm-clerical        -0.086475
occupation_Armed

Defining Fairness Metrics

*Disparate Impact*

In [12]:
def disparate_impact_scorer(y, y_pred, X):

  indexes = y.index.tolist()
  X = X.loc[indexes,:]
  y = y.tolist()
  y_pred = y_pred.tolist()
  
  X['y_pred'] = y_pred
 

  p1 = len(X[(X['y_pred']==1) & (X["gender"] == 0)]) / len(X)
  p2 = len(X[(X["gender"] == 0)]) / len(X)
  p3 = len(X[(X['y_pred']==1) & (X["gender"] == 1)]) / len(X)
  p4 = len(X[(X["gender"] == 1)]) / len(X)

  return min(((p1/p2)/(p3/p4)), ((p3/p4)/(p1/p2)))

*Zemel Fairness*

In [13]:
def zemel_fairness_scorer(y, y_pred, X):

  indexes = y.index.tolist()
  X = X.loc[indexes,:]
  y = y.tolist()
  y_pred = y_pred.tolist()

  X['y'] = y
  X['y_pred'] = y_pred


  p1 = len(X[(X['y_pred']==1) & (X["gender"] == 1)]) / len(X)
  p2 = len(X[(X["gender"] == 1)]) / len(X)

  p3 = len(X[(X['y_pred']==1) & (X["gender"] == 0)]) / len(X)
  p4 = len(X[(X["gender"] == 0)]) / len(X)


  return abs((p1/p2) - (p3/p4))

In [14]:
def BER_scorer(y, y_pred):

  tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

  return (0.5 * ((fn/(fn + tp))+(fp/(fp + tn))))  

# Naive Implementation¶ 

Randomly Splitting Dataset

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
data[data.columns.tolist()] = min_max_scaler.fit_transform(data[data.columns.tolist()])

train_set, test_set = train_test_split(data, test_size=0.3, random_state=42, stratify=data['gender'])

x_train = train_set.drop('income', axis=1)
x_test = test_set.drop('income', axis=1)

y_train = train_set['income']
y_test = test_set['income']

Training Model

In [17]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import RandomizedSearchCV

rfc = RandomForestClassifier(class_weight='balanced_subsample', random_state=42)
rfc.fit(x_train, y_train)

RandomForestClassifier(class_weight='balanced_subsample', random_state=42)

**Evaluating Model Performance**

In [18]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

y_pred = rfc.predict(x_test)

print(f"Test set accuracy = {accuracy_score(y_test, y_pred)}")
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print(f"True pos {tp}")
print(f"False pos {fp}")
print(f"True neg {tn}")
print(f"False neg {fn}")
print(f"Sensitivity = {tp / (tp + fn)}")
print(f"Specificity = {tn / (tn + fp)}")
print(f"Area under ROC curve {roc_auc_score(y_test, y_pred)}")
print(f"Balanced Error Rate: {BER_scorer(y_test, y_pred)}")

Test set accuracy = 0.8561386746741282
True pos 2171
False pos 733
True neg 10374
False neg 1375
Sensitivity = 0.6122391426959954
Specificity = 0.9340055820653642
Area under ROC curve 0.7731223623806799
Balanced Error Rate: 0.22687763761932017


In [19]:
print(f"Disparate Impact: {disparate_impact_scorer(y_test, y_pred, x_test)}")
print(f"Zemel Fairness: {zemel_fairness_scorer(y_test, y_pred, x_test)}")

Disparate Impact: 0.31093386026425535
Zemel Fairness: 0.1769974892879156


# Representative Implementation¶

Resetting Dataset

In [20]:
data = load_data()
data = clean_data(data.copy())

C:\Users\bahab\AppData\Local\Temp\ipykernel_15608\1184947390.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['race'] = data['race'].replace(['White'], [1])
C:\Users\bahab\AppData\Local\Temp\ipykernel_15608\1184947390.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['gender'] = data['gender'].replace(['Female','Male'], [0,1])
C:\Users\bahab\AppData\Local\Temp\ipykernel_15608\1184947390.py:29: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignm

Upsampling minority class

In [21]:
print(data['gender'].value_counts())

gender
1    32650
0    16192
Name: count, dtype: int64


In [23]:
from sklearn.utils import resample

majority = data[(data['gender'] == 1)]
minority = data[(data['gender'] == 0)]

resampled_minority= resample(minority, replace=True, n_samples=((509 * len(majority)) //491), random_state=42)


data = pd.concat([majority, resampled_minority])
data = data.reset_index(drop=True)

In [24]:
print(data['gender'].value_counts())

gender
0    33846
1    32650
Name: count, dtype: int64


In [25]:
data.corrwith(data['income'])

age                             0.215375
fnlwgt                          0.003986
educational-num                 0.316286
race                            0.088630
gender                          0.242525
capital-gain                    0.232173
capital-loss                    0.150589
hours-per-week                  0.222451
native-country                 -0.029708
income                          1.000000
workclass_Federal-gov           0.058175
workclass_Local-gov             0.034724
workclass_Never-worked         -0.006213
workclass_Private              -0.076976
workclass_Self-emp-inc          0.137009
workclass_Self-emp-not-inc      0.038287
workclass_State-gov             0.009492
workclass_Without-pay          -0.007431
marital_status_divorced        -0.131849
marital_status_married          0.467992
marital_status_separated       -0.086741
marital_status_unmarried       -0.301996
marital_status_widowed         -0.066628
occupation_Adm-clerical        -0.093086
occupation_Armed

**Splitting Dataset using Stratifaction by Gender**

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
data[data.columns.tolist()] = min_max_scaler.fit_transform(data[data.columns.tolist()])

train_set, test_set = train_test_split(data, test_size=0.3, random_state=42, stratify=data['gender'])

x_train = train_set.drop('income', axis=1)
x_test = test_set.drop('income', axis=1)

y_train = train_set['income']
y_test = test_set['income']

**Training Model**

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

rfc = RandomForestClassifier(class_weight='balanced_subsample', random_state=42)
rfc.fit(x_train, y_train)

RandomForestClassifier(class_weight='balanced_subsample', random_state=42)

**Evaluating Model Performance**

In [29]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

y_pred = rfc.predict(x_test)

print(f"Test set accuracy = {accuracy_score(y_test, y_pred)}")
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print(f"True pos {tp}")
print(f"False pos {fp}")
print(f"True neg {tn}")
print(f"False neg {fn}")
print(f"Sensitivity = {tp / (tp + fn)}")
print(f"Specificity = {tn / (tn + fp)}")
print(f"Area under ROC curve {roc_auc_score(y_test, y_pred)}")
print(f"Balanced Error Rate: {BER_scorer(y_test, y_pred)}")

Test set accuracy = 0.9061607098100155
True pos 2898
False pos 660
True neg 15179
False neg 1212
Sensitivity = 0.7051094890510949
Specificity = 0.9583307026958773
Area under ROC curve 0.831720095873486
Balanced Error Rate: 0.16827990412651392


**Evaluating Model Fairness**

In [30]:
print(f"Disparate Impact: {disparate_impact_scorer(y_test, y_pred, x_test)}")
print(f"Zemel Fairness: {zemel_fairness_scorer(y_test, y_pred, x_test)}")

Disparate Impact: 0.3935688380733303
Zemel Fairness: 0.15645242942202087


# Fair Implementation

In [31]:
data = load_data()
data = clean_data(data.copy())

C:\Users\bahab\AppData\Local\Temp\ipykernel_15608\1184947390.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['race'] = data['race'].replace(['White'], [1])
C:\Users\bahab\AppData\Local\Temp\ipykernel_15608\1184947390.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['gender'] = data['gender'].replace(['Female','Male'], [0,1])
C:\Users\bahab\AppData\Local\Temp\ipykernel_15608\1184947390.py:29: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignm

**Applying Kamiran and Calders *Classification with No Discrimination***

In [33]:
print(f"male {len(D[(D['gender'] == 1) & (D['income'] == 1)]) / len(D[(D['gender'] == 1)])}")
print(f"female {len(D[(D['gender'] == 0) & (D['income'] == 1)]) / len(D[(D['gender'] == 0)])}")

male 0.20354408225771167
female 0.20352861725476953


**Splitting Dataset using Stratifaction by Gender**

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
D[D.columns.tolist()] = min_max_scaler.fit_transform(D[D.columns.tolist()])

train_set = D

x_train = train_set.drop('income', axis=1)

y_train = train_set['income']

**Training Model**

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

rfc = RandomForestClassifier(class_weight='balanced_subsample', random_state=42)
rfc.fit(x_train, y_train)

RandomForestClassifier(class_weight='balanced_subsample', random_state=42)

**Evaluating Model Performance**

In [36]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

y_pred = rfc.predict(x_test)

print(f"Test set accuracy = {accuracy_score(y_test, y_pred)}")
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print(f"True pos {tp}")
print(f"False pos {fp}")
print(f"True neg {tn}")
print(f"False neg {fn}")
print(f"Sensitivity = {tp / (tp + fn)}")
print(f"Specificity = {tn / (tn + fp)}")
print(f"Area under ROC curve {roc_auc_score(y_test, y_pred)}")
print(f"Balanced Error Rate: {BER_scorer(y_test, y_pred)}")

Test set accuracy = 0.8568850568950824
True pos 2595
False pos 1340
True neg 14499
False neg 1515
Sensitivity = 0.6313868613138686
Specificity = 0.9153986994128417
Area under ROC curve 0.7733927803633551
Balanced Error Rate: 0.22660721963664482


**Evaluating Model Fairness**

In [38]:
print(f"Disparate Impact: {disparate_impact_scorer(y_test, y_pred, x_test)}")
print(f"Zemel Fairness: {zemel_fairness_scorer(y_test, y_pred, x_test)}")

Disparate Impact: 0.9728181413559247
Zemel Fairness: 0.005434230160278997


In [42]:
# Dataset principal cleané
data_clean = clean_data(load_data().copy())
data_clean.to_csv('adult_clean.csv', index=False)

# Train fair (après Kamiran)
D.to_csv('train_fair.csv', index=False)

print("✅ adult_clean.csv + train_fair.csv sauvés")

C:\Users\bahab\AppData\Local\Temp\ipykernel_15608\1184947390.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['race'] = data['race'].replace(['White'], [1])
C:\Users\bahab\AppData\Local\Temp\ipykernel_15608\1184947390.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['gender'] = data['gender'].replace(['Female','Male'], [0,1])
C:\Users\bahab\AppData\Local\Temp\ipykernel_15608\1184947390.py:29: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignm

✅ adult_clean.csv + train_fair.csv sauvés


In [43]:
pd.read_csv('adult_clean.csv').shape

(48842, 43)

In [44]:
df=pd.read_csv('adult_clean.csv')

In [45]:
df.head()

,age,fnlwgt,educational-num,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,...,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife
0,25,226802,7,0,1,0,0,40,0,0,...,False,False,False,False,False,False,False,True,False,False
1,38,89814,9,1,1,0,0,50,0,0,...,False,False,False,False,True,False,False,False,False,False
2,28,336951,12,1,1,0,0,40,0,1,...,True,False,False,False,True,False,False,False,False,False
3,44,160323,10,0,1,7688,0,40,0,1,...,False,False,False,False,True,False,False,False,False,False
4,18,103497,10,1,0,0,0,30,0,0,...,False,False,False,False,False,False,False,True,False,False


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 43 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   age                           48842 non-null  int64
 1   fnlwgt                        48842 non-null  int64
 2   educational-num               48842 non-null  int64
 3   race                          48842 non-null  int64
 4   gender                        48842 non-null  int64
 5   capital-gain                  48842 non-null  int64
 6   capital-loss                  48842 non-null  int64
 7   hours-per-week                48842 non-null  int64
 8   native-country                48842 non-null  int64
 9   income                        48842 non-null  int64
 10  workclass_Federal-gov         48842 non-null  bool 
 11  workclass_Local-gov           48842 non-null  bool 
 12  workclass_Never-worked        48842 non-null  bool 
 13  workclass_Private             4